In [1]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.callbacks import EarlyStopping
from keras.datasets import imdb

2022-04-25 12:24:57.160318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 12:24:57.160365: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
n_words = 1000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=n_words)
print('Train seq: {}'.format(len(X_train)))
print('Test seq: {}'.format(len(X_train)))

Train seq: 25000
Test seq: 25000


In [3]:
print('Train example: \n{}'.format(X_train[0]))
print('\nTest example: \n{}'.format(X_test[0]))

# Note: the data is already preprocessed (words are mapped to vectors)

Train example: 
[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]

Test example: 
[1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 2, 394, 354, 4, 123, 9, 2, 2, 2

In [4]:
# Pad sequences with max_len
max_len = 200
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [5]:
# Define network architecture and compile
model = Sequential()
model.add(Embedding(n_words, 50, input_length=max_len))
model.add(Dropout(0.5))
model.add(Conv1D(128, 3, padding='valid', activation='relu', strides=1,))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           50000     
                                                                 
 dropout (Dropout)           (None, 200, 50)           0         
                                                                 
 conv1d (Conv1D)             (None, 198, 128)          19328     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 250)               32250     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                        

2022-04-25 12:25:00.965610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-25 12:25:00.965653: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-25 12:25:00.965684: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Willy): /proc/driver/nvidia/version does not exist
2022-04-25 12:25:00.965941: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3)]

In [7]:
batch_size = 64
n_epochs = 100

model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, validation_split=0.2, callbacks=callbacks)

Epoch 1/100
313/313 [==============================] - 4s 12ms/step - loss: 0.5640 - accuracy: 0.6844 - val_loss: 0.4124 - val_accuracy: 0.8142
Epoch 2/100
313/313 [==============================] - 3s 11ms/step - loss: 0.3903 - accuracy: 0.8256 - val_loss: 0.3419 - val_accuracy: 0.8522
Epoch 3/100
313/313 [==============================] - 4s 11ms/step - loss: 0.3523 - accuracy: 0.8466 - val_loss: 0.3306 - val_accuracy: 0.8572
Epoch 4/100
313/313 [==============================] - 3s 10ms/step - loss: 0.3295 - accuracy: 0.8592 - val_loss: 0.3211 - val_accuracy: 0.8648
Epoch 5/100
313/313 [==============================] - 4s 11ms/step - loss: 0.3114 - accuracy: 0.8677 - val_loss: 0.3163 - val_accuracy: 0.8652
Epoch 6/100
313/313 [==============================] - 3s 11ms/step - loss: 0.3004 - accuracy: 0.8738 - val_loss: 0.3158 - val_accuracy: 0.8616
Epoch 7/100
313/313 [==============================] - 3s 11ms/step - loss: 0.2933 - accuracy: 0.8759 - val_loss: 0.3119 - val_accuracy:

In [8]:
print('\nAccuracy on test set: {}'.format(model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 1s 975us/step - loss: 0.3017 - accuracy: 0.8731

Accuracy on test set: 0.8731200098991394
